# Assignment 7

Grupo 5: Karla Chauca, Alejandra Navarro, Adriana Sierra y Elisa Vivar

In [ ]:
#pip install openpyxl

In [ ]:
import openpyxl

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import chardet
from shapely.ops import unary_union

In [ ]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import folium 
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap

In [ ]:
# Getting the character format (encoding type)

base = open(r'../../_data/data_dengue_peru.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

## Casos de Dengue

In [ ]:
cv_data = pd.read_csv( r'../../_data/data_dengue_peru.csv', encoding = charenc)
cv_data.head()

In [ ]:
# Limpiar los valores eliminando los separadores de miles
cv_data['Casos'] = cv_data['Casos'].astype(str).str.replace(',', '', regex=False)

# Convertir la columna a valores numéricos, manteniendo NA
cv_data['Casos'] = pd.to_numeric(cv_data['Casos'], errors='coerce')

In [ ]:
# Función para convertir a str y agregar un 0 si la longitud es 5
def format_ubigeo(ubigeo):
    if pd.isna(ubigeo):
        return ubigeo  # Mantener NaN como está
    ubigeo_str = str(ubigeo)
    if len(ubigeo_str) == 5:
        return '0' + ubigeo_str
    return ubigeo_str

# Aplicar la función a la columna 'Ubigeo'
cv_data['Ubigeo'] = cv_data['Ubigeo'].apply(format_ubigeo)

cv_data['ubigeo_dep'] = cv_data['Ubigeo'].str[:2]
cv_data['ubigeo_prov'] = cv_data['Ubigeo'].str[:4]

cv_data = cv_data.rename({'Ubigeo':'ubigeo'}, axis =1 )

# Verificar el df de casos de dengue
cv_data

## Shapefiles

In [ ]:
# Importar el shapefile a nivel de distrito
maps = gpd.read_file(r'../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp')

In [ ]:
# Agregar dos nuevas columnas, con los ubigeos de departamento y provincia

maps = maps.rename({'UBIGEO':'ubigeo'}, axis =1 )

maps['ubigeo_dep'] = maps['ubigeo'].str[:2]
maps['ubigeo_prov'] = maps['ubigeo'].str[:4]
maps = maps[['ubigeo','ubigeo_dep','ubigeo_prov','geometry']]
print(maps.crs)

In [ ]:
# Agrupar por provincia
gdf_prov = maps.groupby('ubigeo_prov').agg({
    'geometry': lambda x: unary_union(x) 
}).reset_index()

# Convertir a GeoDataFrame
gdf_prov = gpd.GeoDataFrame(gdf_prov, geometry='geometry')

# Asegurarse de que el CRS está definido
gdf_prov.set_crs(maps.crs, inplace=True)

# Verificar el GeoDataFrame resultante
print(gdf_prov.crs)

# Shapefile de provincia: gdf_prov

In [ ]:
# Agrupar por departamento
gdf_dep = maps.groupby('ubigeo_dep').agg({
    'geometry': lambda x: unary_union(x) 
}).reset_index()

# Convertir a GeoDataFrame
gdf_dep = gpd.GeoDataFrame(gdf_dep, geometry='geometry')

# Asegurarse de que el CRS está definido
gdf_dep.set_crs(maps.crs, inplace=True)

# Verificar el GeoDataFrame resultante
print(gdf_dep.crs)

# Shapefile de departamento: gdf_dep

In [ ]:
gdf_dist=maps[['ubigeo','geometry']]
print(gdf_dist.crs)

# Shapefile de distrito: gdf_dist

In [ ]:
# Gráficos de los mapas obtenidos
fig, axs = plt.subplots(1, 3, figsize=(11, 5), sharex=True, sharey=True)

# Graficar el GeoDataFrame de distritos
gdf_dist.plot(ax=axs[0], edgecolor='lightgrey', color='black')
axs[2].set_title('Departamentos')

# Graficar el GeoDataFrame de provincias
gdf_prov.plot(ax=axs[1], edgecolor='lightgrey', color='black')
axs[1].set_title('Provincias')

# Graficar el GeoDataFrame de departamentos
gdf_dep.plot(ax=axs[2], edgecolor='lightgrey', color='black')
axs[0].set_title('Distritos')

# Ajustar el espaciado entre subgráficas
plt.tight_layout()

# Mostrar la figura
plt.show()

## Año 2021 - Caso por distrito

In [ ]:
# Agrupar el número de casos por distrito, para todos los años
grouped1 = cv_data.groupby(['ubigeo', 'Año']).agg({
    'Casos': 'sum', # Sumar los casos
    'Distrito': 'first'
}).reset_index()

# Mantener solo los datos de 2021
df1 = grouped1[grouped1['Año'] == 2021]
df1 = df1.reset_index(drop=True)

In [ ]:
# Merge con el shapefile de distrito
df_dist = pd.merge(gdf_dist, df1, how="outer", on=["ubigeo"])

df_dist = gpd.GeoDataFrame(df_dist, geometry='geometry')
df_dist.set_crs(epsg=4326, inplace=True)
print(df_dist.crs)

In [ ]:
#Gráfico

# Máscara para valores faltantes
mask_na = df_dist['Casos'].isna()

fig, ax = plt.subplots(figsize=(10, 10))

# Graficar los valores no nulos con una leyenda continua
df_dist[~mask_na].plot(column='Casos', cmap='RdPu', edgecolor='gray', ax=ax, legend=True, linestyle='-')

# Graficar los valores NaN en color diferente
df_dist[mask_na].plot(edgecolor='white', ax=ax, color='lightgray', linestyle='-')

# Crear la leyenda para los valores NaN
handles, labels = ax.get_legend_handles_labels()
handles.append(plt.Line2D([0], [0], color='lightgray', linewidth=2, linestyle='-', label='No Datos'))
labels.append('Sin datos')

# Configurar la leyenda continua
# Añadir la leyenda continua para los valores no nulos
ax.legend(handles=handles, labels=labels, title='Número de Casos', loc='lower left', bbox_to_anchor=(0, 0), fontsize='small')

# Agregar el título al mapa
ax.set_title("Número de casos, por distrito (2021)", fontsize=16)

plt.show()

## Año 2021 - Casos por provincia

In [ ]:
# Agrupar el número de casos por provincia, para todos los años
grouped2 = cv_data.groupby(['ubigeo_prov', 'Año']).agg({
    'Casos': 'sum', # Sumar los casos
    'Provincia': 'first'
}).reset_index()

# Mantener solo los datos de 2021
df2 = grouped2[grouped2['Año'] == 2021]
df2 = df2.reset_index(drop=True)

In [ ]:
# Merge con el shapefile de provincia
df_prov = pd.merge(gdf_prov, df2, how="outer", on=["ubigeo_prov"])

df_prov = gpd.GeoDataFrame(df_prov, geometry='geometry')
df_prov.set_crs(epsg=4326, inplace=True)
print(df_prov.crs)

In [ ]:
# Gráfico

# Máscara para valores faltantes
mask_na = df_prov['Casos'].isna()

fig, ax = plt.subplots(figsize=(10, 10))

# Graficar los valores no nulos con una leyenda continua
df_prov[~mask_na].plot(column='Casos', cmap='Oranges', edgecolor='lightgray', ax=ax, legend=True, linestyle='-')

# Graficar los valores NaN en color diferente
df_prov[mask_na].plot(edgecolor='gray', ax=ax, color='lightgray', linestyle='-')

# Crear la leyenda para los valores NaN y añadirla a la leyenda continua
handles, labels = ax.get_legend_handles_labels()
handles.append(mlines.Line2D([0], [0], color='lightgray', linewidth=2, linestyle='-', label='No Datos'))
labels.append('Sin datos')

# Configurar la leyenda continua
ax.legend(handles=handles, labels=labels, title='Número de Casos', loc='lower left', bbox_to_anchor=(0, 0), fontsize='small')

# Agregar el título al mapa
ax.set_title("Número de casos, por provincia (2021)", fontsize=16)

plt.show()

## Casos por departamento

In [ ]:
# Agrupar el número de casos por departamento, para todos los años
grouped3 = cv_data.groupby(['ubigeo_dep', 'Año']).agg({
    'Casos': 'sum', # Sumar los casos
    'Departamento': 'first'
}).reset_index()

In [ ]:
# Crear una lista con los años para cada gdf
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]

# Crear un diccionario para almacenar los GeoDataFrames por año
geo_dataframes = {}

for year in years:
    # Filtrar los datos para el año actual
    df3_year = grouped3[grouped3['Año'] == year].reset_index(drop=True)
    
    # Realizar el merge con el GeoDataFrame de departamentos
    df_dep_year = pd.merge(gdf_dep, df3_year, how="outer", on=["ubigeo_dep"])
    
    # Convertir a GeoDataFrame
    df_dep_year = gpd.GeoDataFrame(df_dep_year, geometry='geometry')
    df_dep_year.set_crs(epsg=4326, inplace=True)
    
    # Almacenar en el diccionario con el año como clave
    geo_dataframes[year] = df_dep_year

# Imprimir CRS para verificar
for year, gdf in geo_dataframes.items():
    print(f"CRS para el año {year}: {gdf.crs}")

In [ ]:
# Crear una leyenda para todos los gráficos
def create_legend():
    # Crear un DataFrame para la leyenda
    legend_handles = []
        
    # Añadir el manejador para los valores NaN
    legend_handles.append(mlines.Line2D([0], [0], color='gray', linewidth=2, linestyle='-', label='Sin datos'))
    
    return legend_handles

# Obtener los años y número de GeoDataFrames
years = list(geo_dataframes.keys())
num_years = len(years)

# Crear la figura y los ejes para los subplots
fig, axs = plt.subplots(3, 3, figsize=(20, 20), sharex=False, sharey=True)

# Aplanar el array de ejes para facilitar el acceso
axs = axs.flatten()

# Crear los manejadores de la leyenda
handles = create_legend()

# Graficar cada GeoDataFrame en el subplot correspondiente
for i, (year, gdf) in enumerate(geo_dataframes.items()):
    ax = axs[i]
    
    # Crear la máscara para los valores NaN en la columna 'Casos'
    if 'Casos' in gdf.columns:
        mask_na = gdf['Casos'].isna()
        
        # Graficar los valores no nulos
        gdf[~mask_na].plot(column='Casos', ax=ax, edgecolor='lightgray', cmap='Purples', legend=True)
        
        # Graficar los valores NaN en color diferente
        gdf[mask_na].plot(ax=ax, edgecolor='gray', color='gray')
        
        # Establecer el título para el subplot
        ax.set_title(f'Año {year}')
        
        # Añadir la leyenda en cada subplot
        ax.legend(handles=handles, title='Número de Casos', loc='lower left', bbox_to_anchor=(0, 0), fontsize='small')
    else:
        ax.text(0.5, 0.5, f'No datos para el año {year}', horizontalalignment='center', verticalalignment='center', fontsize=12, color='red')
        ax.set_title(f'Sin información')

# Ocultar los subplots no utilizados
for j in range(num_years, len(axs)):
    axs[j].axis('off')

# Ajustar el espaciado entre subgráficas
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajustar el rectángulo para dar espacio al título

# Agregar el título general en la parte superior
fig.suptitle("Número de casos por departamento (2015-2021)", fontsize=22, y=1.02)

# Mostrar la figura
plt.show()


## Trimestres 2021 - Casos por departamento